Tutorial for using Python with MongoDB for querying (also in MongoDB syntax). Weiterführende Tutorials:

*   https://www.mongodb.com/blog/post/getting-started-with-python-and-mongodb
*   https://dzone.com/articles/getting-started-with-python-and-mongodb
*   http://zetcode.com/python/pymongo/
*   https://www.sitepoint.com/using-joins-in-mongodb-nosql-databases/



In [ ]:
!pip install dnspython

     |████████████████████████████████| 215kB 6.7MB/s 


Check white list in Network Access in Mongo Atlas, you might get a connection error else when querying (not connecting...). Below the command to show which ip address this server has from the outside.

In [ ]:
!curl ipecho.net/plain

34.66.77.46

Connect to the Mongo Atlas, important replace USERNAME and PASSWORD appropriately (Project 0/Database Access/User).

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb+srv://<USERNAME>:<PASSWORD>@cluster0.rndqo.mongodb.net/sample_analytics')

We use the sample_analytics connection.

In [ ]:
db = client.sample_analytics

With the following code you can check the type and which attributes/functions you can do with the object/instance "db".


In [ ]:
print(type(db),dir(db))

<class 'pymongo.database.Database'> ['_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_Database__client', '_Database__incoming_copying_manipulators', '_Database__incoming_manipulators', '_Database__name', '_Database__outgoing_copying_manipulators', '_Database__outgoing_manipulators', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply_incoming_copying_manipulators', '_apply_incoming_manipulators', '_command', '_create_or_update_user', '_current_op', '_default_role', '_fix_incoming', '_fix_outgoing', '_list_collections', '_read_preference_for', '_retryable

Let's list which collections are in this database

In [ ]:
db.list_collection_names()

['customers', 'accounts', 'transactions']

Just checking the status of the server and connections.

In [ ]:
client.admin.command("serverStatus")

{'$clusterTime': {'clusterTime': Timestamp(1606818411, 1),
  'signature': {'hash': b'{y\xadi.p\xeb\x13\x15rLo\xafQD\xc7\xa6\x12\x1dg',
   'keyId': 6880201432511807491}},
 'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 0, 'warning': 0},
 'atlasVersion': {'gitVersion': '4719b6d34c5fbbf73e93f1ff22641ebf02da3249',
  'version': '20201013.0.0.1602254538'},
 'connections': {'available': 490, 'current': 10, 'totalCreated': 70},
 'extra_info': {'note': 'fields vary by platform', 'page_faults': 0},
 'host': 'cluster0-shard-00-02.rndqo.mongodb.net:27017',
 'localTime': datetime.datetime(2020, 12, 1, 10, 26, 51, 350000),
 'mem': {'bits': 64,
  'mapped': 0,
  'mappedWithJournal': 0,
  'resident': 0,
  'supported': True,
  'virtual': 0},
 'metrics': {'atlas': {'bytesInWrites': 346030080,
   'connectionPool': {'totalCreated': 4810}}},
 'network': {'bytesIn': 349686, 'bytesOut': 7649789, 'numRequests': 1386},
 'ok': 1.0,
 'opcounters': {'command': 1362,
  'delete': 0,
  'getmore': 2,
  'i

Let's get the account collection.

In [ ]:
accounts= db.accounts

Print out the type, check that it is a collection: https://docs.mongodb.com/manual/core/databases-and-collections/.

In [ ]:
type(accounts)

pymongo.collection.Collection

Let's get the data from it.

In [ ]:
result = accounts.find()

Interestingly the result is not a collection: https://docs.mongodb.com/manual/reference/method/db.collection.find/index.html

In [ ]:
type(result)

pymongo.cursor.Cursor

dir will tell you that result( Cursor) has a method next, which can be used to iterate through, let's iterate. Be aware, after iterating, the result will be empty (so you could save the data in a list).

In [ ]:
for i1,res in enumerate(result):
  if i1%100==0:
    print(res)
print(i1)

{'_id': ObjectId('5ca4bbc7a2dd94ee5816239a'), 'account_id': 370583, 'limit': 10000, 'products': ['Brokerage', 'Commodity', 'InvestmentStock']}
{'_id': ObjectId('5ca4bbc7a2dd94ee581625ea'), 'account_id': 642405, 'limit': 10000, 'products': ['Brokerage', 'Commodity', 'InvestmentFund', 'CurrencyService', 'InvestmentStock']}
{'_id': ObjectId('5ca4bbc7a2dd94ee58162581'), 'account_id': 529972, 'limit': 10000, 'products': ['CurrencyService', 'Commodity', 'InvestmentStock', 'Brokerage']}
{'_id': ObjectId('5ca4bbc7a2dd94ee58162460'), 'account_id': 898675, 'limit': 10000, 'products': ['InvestmentStock', 'Derivatives', 'Commodity']}
{'_id': ObjectId('5ca4bbc7a2dd94ee58162414'), 'account_id': 261796, 'limit': 10000, 'products': ['InvestmentStock']}
{'_id': ObjectId('5ca4bbc7a2dd94ee58162583'), 'account_id': 785218, 'limit': 10000, 'products': ['Commodity', 'InvestmentStock']}
{'_id': ObjectId('5ca4bbc7a2dd94ee58162578'), 'account_id': 881424, 'limit': 10000, 'products': ['InvestmentStock', 'Deriva

Not lets use mongodb to acquire the data we need. Here we will aggregate two collections, accounts with transactions. Further, we use the account_id from accounts and link to the foreignField account_id from transactions, similarly as "join on" in sql. We rename that transacgtions_link. We will also selection only those records which have account_id less than 370000. 

In [ ]:
result = accounts.aggregate([
    {
        '$lookup': {
            'from': 'transactions', 
            'localField': 'account_id', 
            'foreignField': 'account_id', 
            'as': 'transactions_link'
        }
    }, {
        '$match': {
            '$expr': {
                '$gt': [
                    370000, '$account_id'
                ]
            }
        }
    }
])

Let's check the result.

In [ ]:
for res in result:
  print(res["account_id"],len(res["transactions_link"][0]["transactions"]))

212024 32
328304 62
136137 51
228290 66
276528 47
353465 74
135185 47
137994 65
198100 66
304914 65
324287 21
278603 83
260499 94
209363 56
299072 46
330318 72
299100 52
255695 17
199962 26
116508 81
199711 69
327942 55
165279 5
141237 45
109139 80
230611 20
369055 64
147575 19
183253 66
327520 36
84115 75
53124 41
101383 28
59715 93
76399 53
98267 36
296866 96
330961 27
202669 61
59768 98
358133 13
337979 35
359211 87
165436 27
331043 98
244662 67
285919 33
292277 70
111213 13
194523 3
77032 81
303388 9
292528 45
103346 70
155327 87
170945 66
264514 95
226641 19
66698 75
62713 44
349536 29
344885 27
261248 33
278866 100
155475 8
323636 27
130717 34
205563 57
183400 1
271186 60
118003 39
143720 79
343230 98
158557 80
87965 58
226398 46
278497 79
166084 73
160912 68
139687 91
126833 27
356905 88
141597 3
54977 15
312740 5
303316 99
341068 9
85228 22
154391 4
369761 19
139582 93
78388 28
312052 27
130514 27
62872 54
66589 57
317556 7
57322 79
129238 43
240640 97
149440 8
175894 96
120556

Same story but we could the results.

In [ ]:
result = accounts.aggregate([
    {
        '$lookup': {
            'from': 'transactions', 
            'localField': 'account_id', 
            'foreignField': 'account_id', 
            'as': 'transactions_link'
        }
    }, {
        '$match': {
            '$expr': {
                '$gt': [
                    370000, '$account_id'
                ]
            }
        }
    }, {
        '$count': 'account_id'
    }
])

In [ ]:
for res in result:
  print(res)

{'account_id': 588}


If you want check the results by putting transactions and accounts into two different pandas dataframes and merging them with sql.
For the NoSQL-report the MongoDB query is needed. Using the syntax in the find is enough. You could double check using the pandas approach though.

**Weiterführende Fragen:**

Kannst du auswählen, welche account_id transactions mit weniger als 30 Elemente haben?
Welche account hat die älteste transactions?
Überlege noch 1-2 komplexe Fragen in diesem Datensatz.